In [23]:
using DataFrames;
using JuMP;
using Chemostat;
using Chemostat.Tools;
using Plots;
plotly();

## Loading Model

In [98]:
S, mets, rxns = Chemostat.v1.Models.EColi.load_ecoli_v1();
println("S $(size(S)), mets $(size(mets)) rxns $(size(rxns))");

S (72, 93), mets (72, 4) rxns (93, 6)


## Test at ξmax

In [132]:
ξmax = 3e6;#3e6
fbares = Chemostat.v1.FBA.fba_chemostat_v1(S, mets, rxns, ξmax, ϕub = cost);
println();
println("Results")
println("ϕ: $(getvalue(fbares.ϕ))");
println("Obj: $(getvalue(fbares.obj))");
println();
print("Non zero transport fluxes: ")
println(filter((x) -> x[2] != 0 ,map((x,y) -> (x,y), rxns[:id][find((row) -> row[:t] > 0, eachrow(rxns))],
    Chemostat.v1.FBA.flux.(fbares, rxns[:id][find((row) -> row[:t] > 0, eachrow(rxns))]))))

Academic license - for non-commercial use only

Results
ϕ: 4.5302129572545585e-6
Obj: 2.4081092745626496e-7

Non zero transport fluxes: Tuple{String,Float64}[("CO2t", -1.98022e-5), ("GLCpts", 8.3e-6), ("H2Ot", -2.94563e-5), ("O2t", 2.14988e-5)]


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] uuid4(::MersenneTwister) at ./deprecated.jl:57
 [3] msg_header at /Users/Pereiro/.julia/v0.6/IJulia/src/msg.jl:18 [inlined]
 [4] msg_pub(::IJulia.Msg, ::String, ::Dict{String,String}, ::Dict{String,Any}) at /Users/Pereiro/.julia/v0.6/IJulia/src/msg.jl:30 (repeats 2 times)
 [5] send_stream(::String) at /Users/Pereiro/.julia/v0.6/IJulia/src/stdio.jl:172
 [6] send_stdio(::String) at /Users/Pereiro/.julia/v0.6/IJulia/src/stdio.jl:130
 [7] (::Base.##302#303{IJulia.#send_stdout,Timer})() at ./event.jl:436
while loading In[132], in expression starting on line 10


In [133]:
ξrange = filter((ξ) -> ξ <= ξmax, Tools.logrange(-3,7; base = 1:0.1:9));
println("ξrange length: $(length(ξrange)), start $(start(ξrange)), last $(last(ξrange))")

ξrange length: 750, start 1, last 3.0e6


## Solving

In [147]:
@time fbaress = [Chemostat.v1.FBA.fba_chemostat_v1(S,mets,rxns,ξ) for ξ in ξrange];

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic lice

 17.597100 seconds (112.56 M allocations: 15.278 GiB, 46.60% gc time)


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] uuid4(::MersenneTwister) at ./deprecated.jl:57
 [3] msg_header at /Users/Pereiro/.julia/v0.6/IJulia/src/msg.jl:18 [inlined]
 [4] msg_pub(::IJulia.Msg, ::String, ::Dict{String,String}, ::Dict{String,Any}) at /Users/Pereiro/.julia/v0.6/IJulia/src/msg.jl:30 (repeats 2 times)
 [5] send_stream(::String) at /Users/Pereiro/.julia/v0.6/IJulia/src/stdio.jl:172
 [6] watch_stream(::Base.PipeEndpoint, ::String) at /Users/Pereiro/.julia/v0.6/IJulia/src/stdio.jl:111
 [7] (::IJulia.##20#22)() at ./task.jl:335
while loading In[147], in expression starting on line 240


## Substrate and Waste

In [135]:
Sid = "glc-D[e]";
Sindx = 35;
Stid = "GLCpts";
Stinx = 49;#50
Wid = "ac[e]";
Windx = 6;
Wtid = "ACt2r";
Wtindx = 6;

## glc-D[e] vs ξ

In [136]:
spdata = Chemostat.v1.FBA.s.(fbaress,Sid,Stid);
sp = plot((ξrange, spdata), xaxis = :log, lw = 2)

## ac[e] vs ξ

In [137]:
wpdata = Chemostat.v1.FBA.s.(fbaress,Wid,Wtid);
wp = plot((ξrange,wpdata), xaxis = :log, lw = 2)

## μ vs ξ

In [148]:
τ = 0.0;#5e-6;
mupdata = Chemostat.v1.FBA.μ.(fbaress, wpdata, τ);
mup = plot((ξrange, mupdata), xaxis = :log, lw =2)

## X vs ξ

In [149]:
Xpdata = Chemostat.v1.FBA.X.(fbaress,mupdata,1)
Xp = plot((ξrange, Xpdata), xaxis = :log, lw = 2)

## X vs D (τ > 0)

In [150]:
X_vs_Dpdata = plot((mupdata, Xpdata),lw = 2)

## X vs D (τ = 0)

In [146]:
τ = 0.0;
mupdatant = Chemostat.v1.FBA.μ.(fbaress, wpdata, τ);
Xpdatant = Chemostat.v1.FBA.X.(fbaress,mupdatant,1);
X_vs_Dpdata = plot((mupdatant, Xpdatant), lw = 2)